In [ ]:
!pip install lingtrain_aligner
!pip install razdel
!pip install dateparser
!pip install sentence_transformers
!pip install more_itertools
!pip install hf_xet
!pip install tensorflow

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
from IPython.utils import io
import torch
import time
from torch import optim
from torch.nn import functional as F
import shutil
import pickle
import os
import re

with io.capture_output() as captured:
  !pip install transformers sentencepiece sentence_transformers xmltodict pypinyin pykakasi

from sentence_transformers import SentenceTransformer, SentencesDataset, LoggingHandler, losses
from sentence_transformers.readers import InputExample
from sentence_transformers.evaluation import SentenceEvaluator
from torch.utils.data import DataLoader
from datetime import datetime
import math

from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from tqdm.notebook import tqdm

In [ ]:
import tensorflow as tf

In [ ]:
from lingtrain_aligner import splitter, aligner, resolver, metrics

In [ ]:
cat_ru_path = 'Cat_ru.txt'
cat_en_path = 'Cat_en.txt'

with open(cat_ru_path, 'r', encoding='utf-8') as f:
    cat_ru = f.read()
    print(cat_ru)

with open(cat_en_path, 'r', encoding='utf-8') as f:
    cat_en = f.read()
    print(cat_en)

В отеле было всего двое американцев. Они не знали ни кого из постояльцев с которыми 
встречались на лестнице по пути в свою комнату. Их комната находилась на втором этаже, 
стороной на море. Из нее так-же был виден городской сад с военным монументом в центре. В 
саду росли высокие пальмы и были расставлены зеленые скамейки. В хорошую погоду там 
всегда находился художник с мольбертом. Художнику нравился вид на пальмы и яркие фасады 
отелей обращенные в сторону сада и моря. Итальянцы прибывавшие из далека устраивали 
паломничество к обелиску. Он был сделан из бронзы и блестел от дождя. Шёл дождь. Струи 
ниспадали с пальмовых крон. Вода наполняла ёмкости по сторонам гравиевых дорожек. Море 
широким фронтом выплескивалось на берег и отступало чтобы снова пойти вперед сквозь 
пелену дождя. Автомобили исчезли, оставив пространство перед монументом пустым. 
Напротив, в двери кафе стоял скучающий офицант и обозревал пустую площадь.
Американская жена стояла у окна и смотрела наружу. Прямо под 

In [ ]:
lang_from = 'en'
lang_to = 'ru'
db_path = "Cat_in_the_Rain.db"

splitted_from = splitter.split_by_sentences(cat_en.split('\n'), lang_from)
splitted_to = splitter.split_by_sentences(cat_ru.split('\n'), lang_to)
aligner.fill_db(db_path, lang_from, lang_to, splitted_from, splitted_to)

In [ ]:
len(splitted_from)

118

In [ ]:
for i in range(len(splitted_to)):
  new_line = '\n'
  print(f'en: {splitted_from[i]}{new_line}ru: {splitted_to[i]}')

en: THERE WERE only two Americans stopping at the hotel .
ru: В отеле было всего двое американцев.
en: They did not know any of the people they passed on the stairs on their way to and from their room .
ru: Они не знали ни кого из постояльцев с которыми встречались на лестнице по пути в свою комнату.
en: Their room was on the second floor facing the sea .
ru: Их комната находилась на втором этаже, стороной на море.
en: It also faced the public gar den and the war monument .
ru: Из нее так-же был виден городской сад с военным монументом в центре.
en: There were big palms and green benches in the public garden .
ru: В саду росли высокие пальмы и были расставлены зеленые скамейки.
en: In the good weather there was always an artist with his easel .
ru: В хорошую погоду там всегда находился художник с мольбертом.
en: Artists liked the way the palms grew and the bright colors of the hotels facing the gardens and the sea .
ru: Художнику нравился вид на пальмы и яркие фасады отелей обращенные 

In [ ]:
splitted_to

['В отеле было всего двое американцев.',
 'Они не знали ни кого из постояльцев с которыми встречались на лестнице по пути в свою комнату.',
 'Их комната находилась на втором этаже, стороной на море.',
 'Из нее так-же был виден городской сад с военным монументом в центре.',
 'В саду росли высокие пальмы и были расставлены зеленые скамейки.',
 'В хорошую погоду там всегда находился художник с мольбертом.',
 'Художнику нравился вид на пальмы и яркие фасады отелей обращенные в сторону сада и моря.',
 'Итальянцы прибывавшие из далека устраивали паломничество к обелиску.',
 'Он был сделан из бронзы и блестел от дождя.',
 'Шёл дождь.',
 'Струи ниспадали с пальмовых крон.',
 'Вода наполняла ёмкости по сторонам гравиевых дорожек.',
 'Море широким фронтом выплескивалось на берег и отступало чтобы снова пойти вперед сквозь пелену дождя.',
 'Автомобили исчезли, оставив пространство перед монументом пустым.',
 'Напротив, в двери кафе стоял скучающий офицант и обозревал пустую площадь.',
 'Американс

In [ ]:
model = AutoModel.from_pretrained("sentence-transformers/LaBSE", from_tf=True)

All TF 2.0 model weights were used when initializing BertModel.

All the weights of BertModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.


In [ ]:
aligner.align_db(db_path,
                model_name="sentence_transformer_multilingual_labse",
                batch_size=30,
                window=10,
                batch_ids=[],
                save_pic=False,
                embed_batch_size=5,
                normalize_embeddings=True,
                show_progress_bar=True,
                shift=0)

Aligning without segments.
tasks amount: 12
batch: 0 (0)


Batches:   0%|          | 0/18 [00:00<?, ?it/s]

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

IndexError: list index out of range

In [ ]:
from lingtrain_aligner import vis_helper

In [ ]:
vis_helper.visualize_alignment_by_db(db_path,
        output_path="alignment_vis.png",
        batch_size=118,
        size=(900,900),
        lang_name_from=lang_from,
        lang_name_to=lang_to,
        batch_ids=[],
        plt_show=True,
        show_info=False)

print("score:", metrics.chain_score(db_path))

In [ ]:
conflicts_to_solve, rest = resolver.get_all_conflicts(db_path, min_chain_length=1, max_conflicts_len=10)

In [ ]:
resolver.show_conflict(db_path, conflicts_to_solve[2])

In [ ]:
steps = 3
model_name="sentence_transformer_multilingual_labse"

for i in range(steps):
    conflicts, rest = resolver.get_all_conflicts(db_path,
                        min_chain_length=2+i,
                        max_conflicts_len=6*(i+1),
                        batch_id=-1)

    resolver.resolve_all_conflicts(db_path, conflicts, model_name, show_logs=False)

    if len(rest) == 0:
        break

In [ ]:
vis_helper.visualize_alignment_by_db(db_path,
        output_path="alignment_vis.png",
        batch_size=500,
        size=(900,900),
        lang_name_from=lang_from,
        lang_name_to=lang_to,
        batch_ids=[],
        plt_show=True,
        show_info=False)

print("score:", metrics.chain_score(db_path))

In [ ]:
from lingtrain_aligner import saver

output_path="/content"

saver.save_plain_text(db_path, os.path.join(output_path, f"corpora_{lang_from}.txt"), direction="from", batch_ids=[])
saver.save_plain_text(db_path, os.path.join(output_path, f"corpora_{lang_to}.txt"), direction="to", batch_ids=[])

saver.save_tmx(db_path, os.path.join(output_path, f"corpora.tmx"), lang_from, lang_to)

**Выравнивание для кумандинского и русского**

In [ ]:
dog_ru_path = 'Dog_ru.txt'
dog_kum_path = 'Dog_kum.txt'

with open(dog_ru_path, 'r', encoding='utf-8') as f:
    dog_ru = f.read()
    print(dog_ru)

with open(dog_kum_path, 'r', encoding='utf-8') as f:
    dog_kum = f.read()
    print(dog_kum)

Когда-то на Алтае жил большой народ. Люди на этой земле были с добрыми помыслами, работящими, ловкими. Кругом на лесах, долинах было много ягод, орехов, съедобных грибов. В реках полно рыб. В лесу много разных зверей и птиц. Народ был с весёлым нравом и жил богато. По праздникам для хозяев священных гор и рек они делали жертвоприношения. Почитали ушедших в иной мир предков. Так же благодарили Бога за хорошую жизнь. Так и проходило время. Но вот однажды наступило тяжёлое время – настали очень жаркие дни. В степи пожелтели зелёные травы, в горах деревья стали сохнуть. На земле стали сохнуть ягоды, грибы. Вода в реках обмелела и высохла. Погибла вся рыба. Из тайги ушли звери, улетели птицы. Наступило голодное время. Стали люди болеть и умирать от голода. Тогда старейшины народа, надев свои белые одежды, собрав людей, поднялись на священные горы и стали просить Бога, чтобы народ остался жить на земле. «Мы жили с твоей помощью богато, весело. Теперь погибаем от зноя и голода. Оставь нас и н

In [ ]:
lang_from = ''
lang_to = 'ru'
db_path = "Cat_bread.db"

splitted_from = splitter.split_by_sentences(cat_en.split('\n'), lang_from)
splitted_to = splitter.split_by_sentences(cat_ru.split('\n'), lang_to)
aligner.fill_db(db_path, lang_from, lang_to, splitted_from, splitted_to)

In [ ]:
len(splitted_from)

118

In [ ]:
for i in range(len(splitted_from)):
  new_line = '\n'
  print(f'en: {splitted_from[i]}{new_line}ru: {splitted_to[i]}')

In [ ]:
splitted_to